In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import vcf

%matplotlib inline
import os
from os.path import exists
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import compress
from pylab import MaxNLocator
from matplotlib.colors import LogNorm
from matplotlib import gridspec
from sklearn.preprocessing import StandardScaler
import sys
import pickle

import Bio
from Bio.Alphabet import IUPAC
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.SeqUtils import GC
from Bio import Phylo

from Bio.Align.Applications import MuscleCommandline
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq
from collections import Counter

#for exporting to Adobe Illustrator
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

# [A] SNVS PER GENE

# [1] *Functions* to get mutation events from branches given a tree

In [3]:
def num_mutations_on_branch_SNV(branch_length_i, num_sites_to_construct_tree, locus_length):
    
    '''
    This function takes a branch length as INPUT and OUTPUTs
    (1) the number of neutral mutations that occur on this branch length 
    by drawing from a poisson distribution and (2) a list of the chromosomal
    positions that are mutated (drawn randomly from across the chromosome).
    '''
    
    # get molecular clock rate (mu_i) to convert branch length into years
    mu_i = np.random.uniform(low=0.3, high=0.6)

    # get t_i, the branch length in years
    t_i = (branch_length_i * num_sites_to_construct_tree) / mu_i

    # get the mutation rate for neutral mutations (nu_i) to calculate lambda
    nu_i = np.random.uniform(low=0.3, high=0.6)
    
    # normalize mutation rate for the length of the gene/locus
    nu_i = nu_i * (float(locus_length)/ 4000000.0)

    # calculate lambda, parameter for poisson distribution
    lambda_i = t_i * nu_i

    # draw from poisson distribution to get the number of neutral mutations that occurred on this branch
    num_mutations_i = np.random.poisson(lam=lambda_i)
    
    # choose number drawn above positions from range(0, length of locus) that were "mutated", choose the H37Rv ref positions that mutated randomly
    positions_mutated_i = np.random.randint(low=1, high=locus_length+1, size=num_mutations_i)
    
    return positions_mutated_i

In [4]:
def get_mutation_events_from_tree_branches(num_sites_to_construct_tree, isolate_group, locus_length_dict, locus_count_dict):

    # Load in PHYLOGENY
    #########################################################################################################
    # We're going to use Biopython's Phylo module to load phylogenetic trees created by Luca
    
    phylogeny_path = '/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/phylogenies/tree_output_files/phylogeny_lineage_' + isolate_group + '/tree_lineage_' + isolate_group + '_iqtree_FINAL.treefile'

    # parses and load tree
    tree = Phylo.parse(phylogeny_path , 'newick').next() 

    # root the tree with the outgroup M. canettii ["Normally you will want the outgroup to be a monophyletic group, rather than a single taxon."]
    tree.root_with_outgroup({"name":"canettii"})

    # flip branches so deeper clades are displayed at top
    tree.ladderize()

    # TERMINAL BRANCHES
    #########################################################################################################
    # retrieves the terminal nodes of the tree
    terminal_nodes = tree.get_terminals()

    # Lengths of terminal branches
    terminal_branch_length_list = [terminal_nodes[i].branch_length for i in range(0 , len(terminal_nodes))][1:]

    # retrieve Sample IDs that were used for this tree
    isolate_tags_in_phylogeny = [terminal_nodes[i].name for i in range(0 , len(terminal_nodes))]
    
    # iterate through each LOCUS & get list of positions mutated
    for locus_i in locus_length_dict.keys():

        # iterate through each branch-length & collect
        # (1) the chromosomal positions of the mutations 
        mutation_positions_across_terminal_branches_list = []

        for branch_length_i in terminal_branch_length_list:

            mutation_positions_per_branch_i = num_mutations_on_branch_SNV(branch_length_i, num_sites_to_construct_tree, locus_length_dict[locus_i])

            mutation_positions_across_terminal_branches_list = mutation_positions_across_terminal_branches_list + list(mutation_positions_per_branch_i)

        # Counter dict for the number of times each chromosomal position is mutated
        mutation_events_per_pos_from_terminal_dict = Counter(mutation_positions_across_terminal_branches_list)
        
        # update this in Counter dict for LOCUS
        locus_count_dict[locus_i] = locus_count_dict[locus_i] + mutation_events_per_pos_from_terminal_dict

    # INTERNAL BRANCHES
    #########################################################################################################
    # retrieves the internal nodes of the tree
    internal_nodes = tree.get_nonterminals() 

    # Lengths of internal branches
    internal_branch_length_list = [internal_nodes[i].branch_length for i in range(0 , len(internal_nodes))][2:]
    
    # iterate through each LOCUS & get list of positions mutated
    for locus_i in locus_length_dict.keys():

        # iterate through each branch-length & collect
        # (1) the chromosomal positions of the mutations 
        mutation_positions_across_internal_branches_list = []

        for branch_length_i in internal_branch_length_list:

            mutation_positions_per_branch_i = num_mutations_on_branch_SNV(branch_length_i, num_sites_to_construct_tree, locus_length_dict[locus_i])

            mutation_positions_across_internal_branches_list = mutation_positions_across_internal_branches_list + list(mutation_positions_per_branch_i)

        # Counter dict for the number of times each chromosomal position is mutated
        mutation_events_per_pos_from_internal_dict = Counter(mutation_positions_across_internal_branches_list)
        
        # update this in Counter dict for LOCUS
        locus_count_dict[locus_i] = locus_count_dict[locus_i] + mutation_events_per_pos_from_internal_dict

# [2] Simulate for each LOCUS and save results of simulations

#### load dictionary with (corrected) lengths for each gene we compute mutational density for

In [5]:
with open('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/mutational_density_simulations_per_locus/locus_length_dict.pickle', 'rb') as handle:
    locus_length_dict = pickle.load(handle)

## [2.1] Simulate mutation events for each tree and aggregate across trees

In [10]:
# input 
sim_i = '1'

In [12]:
isolate_group_list = ['1','2','3','4A','4B','4C','5','6']

# sites used to construct each tree
num_sites_to_construct_tree_dict = {'1':243940.0, '2':243542.0, '3':188741.0, '4A':213804.0, '4B':207271.0, '4C':203590.0, '5':33818.0, '6':36923.0}

# create a Counter dict for each locus to keep track of positions mutated (and now many times)
locus_count_dict = {}
for locus_i in locus_length_dict.keys():
    locus_count_dict[locus_i] = Counter()

for isolate_group_i in isolate_group_list:

    get_mutation_events_from_tree_branches(num_sites_to_construct_tree_dict[isolate_group_i], isolate_group_i, locus_length_dict, locus_count_dict)
    
# iterate through each LOCUS and create a series of the sum of mutations (sum Hs for each position mutated)
Hs_score_per_locus = pd.Series()
for locus_i in locus_count_dict.keys():
    
    Hs_score_per_locus[locus_i] = float(pd.Series(locus_count_dict[locus_i]).sum())

# give series a name according to which simulation # it is
Hs_score_per_locus.name = 'S'+sim_i

# store pandas series for downstream analysis
with open('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/mutational_density_simulations_per_locus/simulation_results/sim_'+sim_i+'.pickle', 'wb') as handle:
    pickle.dump(Hs_score_per_locus, handle, protocol=pickle.HIGHEST_PROTOCOL)

## [3] Submit script for each simulation as job to O2

In [4]:
from slurmpy import Slurm
import os

In [7]:
for sim_i in range(1, 101):

    LOCUS_sims_job = 'python /home/rv76/Farhat_Lab/Python_Scripts/homoplasy_project/simulations_SNVs_per_gene.py ' + str(sim_i)

    #directory where you want output + error files
    os.chdir('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/muts_per_LOCUS_sims_homoplasy_score_jobs')

    job_name = 'LOCUS_sim_' + str(sim_i)

    s = Slurm(job_name , {'partition':'short' , 'account':'farhat', 'N':'1' , 'time':'0-2:00:00' , 'mem':'16G' , 'mail-type':'FAIL' , 'mail-user':'roger_vargas@g.harvard.edu'})

    #submits the job
    job_id = s.run(LOCUS_sims_job)

    print job_name  + ' : ' +  str(job_id)

submitted: Submitted batch job 51970840


LOCUS_sim_1 : 51970840


submitted: Submitted batch job 51970841
submitted: Submitted batch job 51970842
submitted: Submitted batch job 51970843


LOCUS_sim_2 : 51970841
LOCUS_sim_3 : 51970842
LOCUS_sim_4 : 51970843


submitted: Submitted batch job 51970844
submitted: Submitted batch job 51970845
submitted: Submitted batch job 51970846


...

submitted: Submitted batch job 51970942
submitted: Submitted batch job 51970943
submitted: Submitted batch job 51970944


LOCUS_sim_91 : 51970942
LOCUS_sim_92 : 51970943
LOCUS_sim_93 : 51970944


submitted: Submitted batch job 51970945
submitted: Submitted batch job 51970946
submitted: Submitted batch job 51970947
submitted: Submitted batch job 51970948


LOCUS_sim_94 : 51970945
LOCUS_sim_95 : 51970946
LOCUS_sim_96 : 51970947
LOCUS_sim_97 : 51970948
LOCUS_sim_98 : 51970949
LOCUS_sim_99 : 51970950
LOCUS_sim_100 : 51970951


submitted: Submitted batch job 51970949
submitted: Submitted batch job 51970950
submitted: Submitted batch job 51970951


## [4] Load series of homoplasy score counts across each locus for each simulation

In [3]:
Hs_score_per_locus_sims = []
for sim_i in range(1, 101):
    
    with open('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/mutational_density_simulations_per_locus/simulation_results/sim_' + str(sim_i) + '.pickle', 'rb') as handle:
        Hs_score_per_locus = pickle.load(handle)
    
    Hs_score_per_locus_sims.append(Hs_score_per_locus)

In [4]:
Hs_score_per_locus_sims_df = pd.concat(Hs_score_per_locus_sims, axis=1)

In [5]:
Hs_score_per_locus_sims_df.head()

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S91,S92,S93,S94,S95,S96,S97,S98,S99,S100
Rv0239,82.0,89.0,96.0,115.0,107.0,103.0,114.0,85.0,93.0,108.0,...,108.0,109.0,97.0,98.0,93.0,106.0,83.0,102.0,92.0,100.0
Rv0238,249.0,294.0,279.0,268.0,235.0,261.0,244.0,250.0,290.0,253.0,...,267.0,240.0,258.0,288.0,313.0,276.0,288.0,276.0,282.0,289.0
Rv1322,137.0,139.0,120.0,127.0,118.0,139.0,146.0,133.0,120.0,140.0,...,143.0,145.0,119.0,134.0,125.0,125.0,135.0,135.0,138.0,138.0
Rv2928,351.0,341.0,294.0,324.0,334.0,322.0,295.0,336.0,332.0,314.0,...,321.0,310.0,338.0,337.0,313.0,321.0,312.0,337.0,328.0,287.0
Rv1324,379.0,361.0,429.0,359.0,395.0,398.0,391.0,407.0,405.0,390.0,...,428.0,394.0,403.0,417.0,387.0,416.0,370.0,402.0,363.0,398.0


add columns for **gene length**

In [6]:
with open('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/mutational_density_simulations_per_locus/locus_length_dict.pickle', 'rb') as handle:
    locus_length_dict = pickle.load(handle)

In [7]:
gene_length_list = []

for gene_i in Hs_score_per_locus_sims_df.index:
    gene_i_length = locus_length_dict[gene_i]
    gene_length_list.append(gene_i_length)
    
Hs_score_per_locus_sims_df.loc[:,'gene_length'] = gene_length_list

In [8]:
Hs_score_per_locus_sims_df.head()

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S92,S93,S94,S95,S96,S97,S98,S99,S100,gene_length
Rv0239,82.0,89.0,96.0,115.0,107.0,103.0,114.0,85.0,93.0,108.0,...,109.0,97.0,98.0,93.0,106.0,83.0,102.0,92.0,100.0,234
Rv0238,249.0,294.0,279.0,268.0,235.0,261.0,244.0,250.0,290.0,253.0,...,240.0,258.0,288.0,313.0,276.0,288.0,276.0,282.0,289.0,615
Rv1322,137.0,139.0,120.0,127.0,118.0,139.0,146.0,133.0,120.0,140.0,...,145.0,119.0,134.0,125.0,125.0,135.0,135.0,138.0,138.0,297
Rv2928,351.0,341.0,294.0,324.0,334.0,322.0,295.0,336.0,332.0,314.0,...,310.0,338.0,337.0,313.0,321.0,312.0,337.0,328.0,287.0,740
Rv1324,379.0,361.0,429.0,359.0,395.0,398.0,391.0,407.0,405.0,390.0,...,394.0,403.0,417.0,387.0,416.0,370.0,402.0,363.0,398.0,915


In [9]:
x = Hs_score_per_locus_sims_df.mean(axis=1) / Hs_score_per_locus_sims_df.gene_length.astype(float)

In [10]:
x.head(n=20)

Rv0239     0.441863
Rv0238     0.436674
Rv1322     0.436877
Rv2928     0.438908
Rv1324     0.436801
Rv1321     0.438944
Rv0231     0.439721
Rv0233     0.441303
Rv0232     0.436131
Rv2929     0.444267
Rv0042c    0.436828
Rv2352c    0.435433
Rv0965c    0.437082
Rv1323     0.438091
Rv0693     0.436915
Rv1567c    0.438840
Rv3534c    0.437233
Rv3256c    0.437662
Rv0878c    0.436585
Rv0422c    0.437554
dtype: float64

In [11]:
x.min()

0.42326732673267325

In [12]:
x.mean()

0.43736258246365517

In [13]:
x.max()

0.4538613861386138

In [14]:
np.percentile(np.array(x), 99.8)

0.4486131403607093

## [3] Re-run simulation N times

In [14]:
num_sims = 100
homoplasy_score_distribution_per_run = {}
for sim_i in range(1, num_sims+1):

    mutation_events_per_pos_dict = Counter()

    for isolate_group_i in isolate_group_list:

        num_mutations_per_branch_series_i, mutation_events_per_pos_dict_i = get_mutation_events_from_tree_branches(num_sites_to_construct_tree_dict[isolate_group_i], isolate_group_i)
        mutation_events_per_pos_dict = mutation_events_per_pos_dict + mutation_events_per_pos_dict_i

    # distribution of homoplasy score across all mutant alleles
    mutation_events_per_pos_series = pd.Series(mutation_events_per_pos_dict)
    homoplasy_score_per_mutation = pd.Series(Counter(mutation_events_per_pos_series))

    # store distribution of homoplasy scores in dict
    homoplasy_score_distribution_per_run['sim_' + str(sim_i)] = homoplasy_score_per_mutation

### Save results of simulations

In [17]:
with open('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/SNV_homoplasy_score_distribution_simluation_counts.pickle', 'wb') as handle:
    pickle.dump(homoplasy_score_distribution_per_run, handle, protocol=pickle.HIGHEST_PROTOCOL)

To load dictionary of **homoplasy score** counts for each simulation

In [4]:
with open('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/pickled_files/SNV_homoplasy_score_distribution_simluation_counts.pickle', 'rb') as handle:
    homoplasy_score_distribution_per_run = pickle.load(handle)